https://github.com/langchain-ai/rag-from-scratch/blob/main/rag_from_scratch_10_and_11.ipynb

some deviations from the source code because i dont wanna pay for embeddings from openai, or hit openai models. All openAI integration is replaced with ollama.

I also removed langsmith integration. don't think it's needed. just a frontend for LLM debugging which i can achieve with `langchain.debug = True`

For routing i'll deviate quite heavily from the notebook above as the methods applied are quite trivial to implement, will attempt to implement a custom example myself 

## DB routing
In some case, we may want to look up a specific DB based on the question that's provided. Use an LLM to provide the routing.

In [43]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser

In [44]:
db_route_prompt_template = """Given a user's question, determine the most likely type of database (choices are graph database, relational database, nosql database) that should be used to answer his query. If you are not able to determine the database type, respond with 'na'

user_prompt:
{user_prompt}
"""

db_route_prompt = ChatPromptTemplate.from_template(db_route_prompt_template)

In [45]:
# Get chat model
from langchain_ollama.chat_models import ChatOllama
from pydantic import BaseModel
from typing import Literal

class PromptRouteModel(BaseModel):
    answer: Literal['graph database', 'relational database', 'nosql database', 'na']
    reason: str

db_route_llm = ChatOllama(
    model="llama3.2:3b-instruct-q5_K_M",
    temperature=0,
    format=PromptRouteModel.model_json_schema()
)

In [59]:
user_prompt_graph = "I need to run weakly connect component algorithm to figure out a community of users"
user_prompt_relational = 'I need to use sql to look up how many orders a user has made'
user_prompt_nosql = 'I need to do an index look up based on a given index and retrieve a bunch of unparsed data'
user_prompt_na = 'who is mans best friend' # fail case
user_prompt_na2 = 'tell me about gojo satoru'

In [49]:
chain = db_route_prompt | db_route_llm | StrOutputParser()

In [51]:
print(chain.invoke({'user_prompt': user_prompt_graph}))

{
  "answer": "graph database",
  "reason": "The Weakly Connected Component (WCC) algorithm is typically used in graph theory and network analysis. Graph databases are well-suited for storing and querying complex networks, making them a good fit for this use case."
}


In [52]:
print(chain.invoke({'user_prompt': user_prompt_relational}))

{
  "answer": "relational database",
  "reason": "The question involves looking up data from a specific table (orders) and performing a count operation, which is typical of relational databases."
}


In [57]:
print(chain.invoke({'user_prompt': user_prompt_nosql}))

{"answer": "nosql database", "reason": "Index lookup is a common operation in NoSQL databases like MongoDB or Cassandra. Relational databases are not optimized for this type of query, while graph databases might be overkill for simple index lookups."}


In [58]:
# lol ok. the answer makes sense but still funny.
print(chain.invoke({'user_prompt': user_prompt_na}))

{
  "answer": "graph database",
  "reason": "The question asks about a specific relationship ('best friend') which is often modeled as a graph in databases. Graph databases are well-suited for modeling complex relationships between entities."
}


In [60]:
print(chain.invoke({'user_prompt': user_prompt_na2}))

{
  "answer": "na",
  "reason": "The question is too open-ended and doesn't contain specific keywords related to a particular database structure. It could be answered using various types of databases."
}


## Prompt routing

Similar idea as DB routing, except instead of deciding on DB, you decide on different prompts. The code will be almost identical to above, so no point redoing it.